In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# maybe put a list here of all links/find a way to automate?

# TEST RANDOM ORG
url = 'https://unt.campuslabs.com/engage/organization/3dprinting'
res = requests.get(url, headers=headers)
try:
    res.raise_for_status()
except Exception as e:
    print(f'problem occured: {e}')

soup = BeautifulSoup(res.text, 'html.parser')

script_tag = soup.find('script', string=re.compile("window.initialAppState"))

if script_tag:
    json_txt = script_tag.string.strip()
    json_txt = json_txt.replace('window.initialAppState = ', '').rstrip(';')

    data = json.loads(json_txt)
    org = data.get('preFetchedData', {}).get('organization', {})
    name = org.get('name')
    short_name = org.get('shortName')
    summary = org.get('summary')
    desc_raw = org.get('description', '')

    desc_clean = BeautifulSoup(desc_raw, 'html.parser').get_text(separator=' ')
    desc_clean = re.sub(r'https?://\S+', '', desc_clean)

    print(f'Name: {name}\nShortname: {short_name}\nSummary: {summary}\nDescription: {desc_clean}')
else:
    raise Exception('script_tag not found')

Name: 3D Printing Club
Shortname: None
Summary: Learn about, design, use, and maintain additive manufacturing tools while learning and working together. Maintain a focus on sustainability and recycling while continuously learning and improving our craft.
Description: This is the 3D printing Club! We are a group of passionate individuals interested in building and using 3D printers. One of our main focuses is sustainability and recycling. Our goal is to eventually be able to recycle bottles into filament, and grind old prints into filament to be reused. Our first printer project is made from the scrapped frame of an old printer once used here at UNT. Any major can join! We are looking for anyone interesting in any aspect of the 3D printing industry, from fashion, marketing, logistics, engineering, ceramics, graphic design, and all sorts of other activities.  
 Join our discord server here:  


In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import re
import sqlite3

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

links = []
try:
    with open('db_and_csvs/unt_club_links.csv', 'r') as rfile:
        for link in rfile:
            link = link.strip()
            links.append(link)
except Exception as e:
    print(f'file error occured: {e}')

conn = sqlite3.connect('db_and_csvs/unt_clubs.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS organizations (
        name TEXT,
        short_name TEXT,
        summary TEXT,
        description TEXT,
        picture TEXT,
        url TEXT           
    )
''')

# FIX LOOP CONDITION?
# cntr = 0
for urls in links:
    url = str(urls)
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()

        soup = BeautifulSoup(res.text, 'html.parser')

        script_tag = soup.find('script', string=re.compile("window.initialAppState"))

        if script_tag:
            json_txt = script_tag.string.strip()
            json_txt = json_txt.replace('window.initialAppState = ', '').rstrip(';')

            data = json.loads(json_txt)
            org = data.get('preFetchedData', {}).get('organization', {})
            name = org.get('name')
            short_name = org.get('shortName')
            summary = org.get('summary')
            desc_raw = org.get('description') or ""
            image_path = org.get('profilePicture')
            picture = f'https://se-images.campuslabs.com/clink/images/{image_path}?preset=med-sq'

            desc_clean = BeautifulSoup(desc_raw, 'html.parser').get_text(separator=' ')
            desc_clean = re.sub(r'https?://\S+', '', desc_clean)

            org_info = {
                'name': name,
                'short_name': short_name,
                'summary': summary,
                'description': desc_clean,
                'picture': picture,
                'url': url

            }

            # COMMENT ME?
            # print(f'Name: {name}\nShortname: {short_name}\nSummary: {summary}\nDescription: {desc_clean}')
            # print("\n")

            cursor.execute('''
                INSERT OR REPLACE INTO organizations (name, short_name, summary, description, picture, url) 
                VALUES (:name, :short_name, :summary, :description, :picture, :url)
            ''', org_info)
            conn.commit()
            # cntr = cntr+1

        else:
            print('script_tag not found')

    except Exception as e:
        print(f'error at {url}: {e}')
        continue

    # print(f'lines: {cntr}')
conn.close()
print('db closed')

db closed


In [2]:
conn.close()